In [ ]:
# This notebook just tests image display

In [ ]:
import lsst.afw.display as afwDisplay
import lsst.daf.persistence as dafPersist
import os
from matplotlib import pyplot as plt
%matplotlib inline
import time
import numpy as np

from lsst.ts import salobj
import asyncio
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observing.utilities.auxtel.latiss.utils import parse_visit_id, parse_obs_id

In [ ]:
# for tab completion to work
%config IPCompleter.use_jedi = False

In [ ]:
from lsst.ts import salobj
from lsst.ts import idl
print(salobj.__file__)
print(idl.__file__)
print(salobj.__version__)
print(idl.__version__)

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
#get classes and start them
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
latiss = LATISS(domain)
await asyncio.gather(latiss.start_task)

In [ ]:
await latiss.enable()

In [ ]:
tmp = await latiss.rem.atspectrograph.evt_heartbeat.next(flush=True)

In [ ]:
tmp = await latiss.rem.atspectrograph.cmd_start.start()

In [ ]:
tmp = await salobj.set_summary_state(latiss.rem.atspectrograph, salobj.State.ENABLED)

In [ ]:
tmp = latiss.rem.atarchiver.evt_summaryState.get()
print(salobj.State(tmp.summaryState))

In [ ]:
# # Define a colon (:) separated playlist - files put in by Tony
tmp = await latiss.rem.atcamera.cmd_playlist.set_start(playlist='test_take_sequence.dat', 
                                                       images='AT_O_20210218_000252',#:AT_O_20210218_000259', 
                                                       folder='emu2')

In [ ]:
# # # Define a playlist - files put in by me
# tmp = await latiss.rem.atcamera.cmd_playlist.set_start(playlist='test_pi.dat', #'latissAcquireAndTakeSequence_test_take_sequence.dat', 
#                                                        images='AT_O_20210121_000739:AT_O_20210121_000740:AT_O_20210121_000741:AT_O_20210121_000742:AT_O_20210121_000743', 
#                                                        folder='script_test_data3')

In [ ]:
# # Define a playlist - files put in by me
# tmp = await latiss.rem.atcamera.cmd_playlist.set_start(playlist='test.dat', #'latissAcquireAndTakeSequence_test_take_sequence.dat', 
#                                                        images='AT_O_20210121_000739     AT_O_20210121_000740',# AT_O_20210121_000741, AT_O_20210121_000742, AT_O_20210121_000743', 
#                                                        folder='script_test_data')

In [ ]:
# Load a playlist
tmp = await latiss.rem.atcamera.cmd_play.set_start(playlist='test_take_sequence.dat', repeat=True)
#tmp = await latiss.rem.atcamera.cmd_play.set_start(playlist='test_pi.dat')"

In [ ]:
#tmp = await latiss.rem.atcamera.cmd_takeImages.set_start(numImages=1, expTime=1, keyValueMap='groupId: asdfasdf, imageType:object')

In [ ]:
tmp2 = latiss.rem.atcamera.evt_endReadout.flush()
tmp2 = latiss.rem.atarchiver.evt_imageInOODS.flush()
exptime=1
tmp = await latiss.take_object(exptime=exptime)
print('done')

In [ ]:
evt = await latiss.rem.atarchiver.evt_imageInOODS.next(flush=False, timeout=exptime+10)
print(evt)

In [ ]:
evt.obsid

In [ ]:
# Want to display in firefly?
afwDisplay.setDefaultBackend('firefly')
os.environ['FIREFLY_HTML'] = "slate.html"
os.environ['FIREFLY_URL'] = 'http://139.229.170.210:8080/firefly/'
#os.environ['FIREFLY_URL'] = 'https://lsst-nts-k8s.ncsa.illinois.edu/portal/app/' # 'http://139.229.170.210:8080/firefly/'

In [ ]:
plt.close('all')
disp = afwDisplay.Display(2, reopenPlot=True, verbose=True, name='PatrickFireFly')
# Print the link
disp.getClient().display_url()

In [ ]:
#datapath = '/project/shared/auxTel/' # for summit images
datapath = '/lsstdata/offline/teststand/NCSA_auxTel/gen2repo'
repo = os.path.join(datapath)#, mapper={'calibRoot': "/home/saluser/ingest/dmcs/CALIB"})
butler = dafPersist.Butler(repo)

In [ ]:
# # # Custom frame
# dayObs = "2021-02-18"
# seqNum = "212"
# dataId = dict(dayObs=dayObs, seqNum=seqNum)
# dataset='raw'
# before = time.time()
# exp = butler.get(dataset, dataId)
# after = time.time()
# print(f'Time to run was: {after-before:0.1f} seconds')

In [ ]:
from lsst.rapid.analysis import BestEffortIsr
bestEffort = BestEffortIsr(datapath)
bestEffort.writePostIsrImages = False  # Don't write to butler database

In [ ]:
# dataId=parse_visit_id(tmp[0])
source, controller, dayObs, seqNum =parse_obs_id(evt.obsid)
dataId=dict(dayObs=dayObs, seqNum=seqNum)
print(dataId)

In [ ]:
before = time.time()
dataset='raw'
exp = butler.get(dataset, dataId)
#exp = bestEffort.getExposure(dataId)
after = time.time()
print(f'Time to run was: {after-before:0.1f} seconds')

In [ ]:
exp.image.array

In [ ]:
plt.imshow(exp.image.array)
#plt.show()

In [ ]:
import numpy as np
med=np.median(exp.image.array)
print(f'Median is {med} ADU')

In [ ]:
p1=(2500,2500)
p2=(2600,2600)
print(f'pixels 2500,2500 and 2600,2600 are {exp.image.array[p1]} and {exp.image.array[p2]}')

In [ ]:
print(f'p2-p1 is {exp.image.array[p2]-exp.image.array[p1]}')

In [ ]:
import struct
def double_to_hex(f):
    return hex(struct.unpack('<Q', struct.pack('<d', f))[0])

In [ ]:
print(double_to_hex(exp.image.array[p2]))
print(double_to_hex(exp.image.array[p1]))

In [ ]:
# hdr0=exp.getInfo()
# hdr_meta=exp.getMetadata()
# print(hdr_meta)

In [ ]:
if True: # display the image in firefly
    #plt.close('all')
    #disp = afwDisplay.Display(2, reopenPlot=True)
    disp.setMaskPlaneColor('SAT', afwDisplay.IGNORE)
    disp.setImageColormap('gray')
    disp.scale('linear', 'zscale')
    disp.mtv(exp, title='dayobs = {}, seqNum = {}'.format(dayObs,seqNum))
    #cgUtils.overlayCcdBoxes(isr_corr_exposure.getDetector(), isTrimmed=True, display=disp)

In [ ]:
import time
import lsst.daf.persistence as dafPersist
butler = dafPersist.Butler('/project/shared/auxTel/')
dataId = {'dayObs': '2020-03-12', 'seqNum': 197}
for i in range(10):
    before = time.time()
    raw = butler.get('raw', **dataId)
    dt = time.time() - before
    print(f'Time = {dt:.2f}s')

In [ ]:
tmp = butler.get('raw_filename', **dataId)

In [ ]:
tmp

In [ ]:
2*3.14*(0.03)*30